## Hierarchical Architecture

In [28]:
from dotenv import load_dotenv
load_dotenv()

True

In [29]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage 
from langchain.chat_models import ChatOpenAI

from langgraph.graph import END, StateGraph

import functools
import operator
from typing import List, Sequence, TypedDict, Annotated
import json
import os

from IPython.display import Image, display

In [30]:
unique_id = "Hierarchical Optimisation"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

In [31]:
from langsmith import Client

client = Client()

In [32]:
class PromptReview(BaseModel):
    """Review of the prompt"""
    feedback: str = Field(description="Feedback on the most recent prompt")


class WorkerAgent:
    """
    Worker Agent class defining agents that provide feedback on prompts.
    """

    def __init__(self, position: str, role: str, function: str, temp: float = 0.0, model: str = "gpt-4o"):
        self.position = position
        self.role = role
        self.function = function
        self.system_message = SystemMessage(content=f"""You are an expert: {self.position}. Your role: {self.role}. Your function: {self.function}.
You must use your expertise to guide all your thinking. You must speak only as an expert in your field.""")        
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )

    def generate(self, prompt: str, additional_info: str) -> PromptReview:
        """
        Generates a review of the prompt.
        """
        template = """I'm going to tip $300K for feedback that results in the most improvement to the prompt.
Your task is to provide detailed feedback and recommendations on the prompt below.
Think about how you can utilise the skills that come with your position, role and function to write a prompt good at eliciting the desired response.

### Prompt: {prompt}

{additional_info}
Only provide feedback and recommendations for the prompt.
Your feedback must be less than 50 words so think carefully about the most critical aspects of the prompt that need improvement.
Do not provide feedback regarding the placeholder text and do not make assumptions on what the placeholders represent.

Return only the original prompt and your feedback in JSON fromat below:

{{
    "feedback": "Feedback on the original prompt"
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=PromptReview)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["position", "role", "function", "prompt", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                completion = chain.invoke({"position": self.position, "role": self.role, "function": self.function, "prompt": prompt, "additional_info": additional_info})
                # Validate the output before returning
                if completion.feedback:
                    return completion
                else:
                    print("Validation failed: Missing required fields in completion")
                    print("Raw output:", completion)
            except Exception as e:
                print("Exception occurred:", e)
                continue
        else:
            raise Exception("Failed to parse output after 3 attempts")

In [33]:
class Workforce(BaseModel):
    """Details of workforce generated by the leader agent."""
    positions: List[str] = Field(description="Positions of the workers in the workforce")
    roles: List[str] = Field(description="Roles of the workers in the workforce")
    functions: List[str] = Field(description="Functions of the workers in the workforce")
    

class FeedbackSummary(BaseModel):
    """Summary of feedback from the worker agents."""
    feedback_summary: str = Field(description="Collated and summarised feedback from the workforce")


class TeamLeaderAgent:
    """
    TeamLeaderAgent class defining an agent that manages a team of worker agents to help optimise prompts.
    """

    def __init__(self, team: str, prompt: str, additional_info: str = None, temp: float = 0.0, model: str = "gpt-4o", workforce: List[WorkerAgent] = None):
        self.team = team
        self.system_message = SystemMessage(content=f"""You are an experienced {team} team leader with expertise in {team}. 
You must use your expertise to guide all your thinking. You must speak only as an expert in your field.""")
        self.prompt = prompt
        self.additional_info = additional_info
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )
        self.workforce = workforce if workforce else self.generate_workforce()
        self.feedback = []
        self.updates = 0
        self.graph = None

    def generate_workforce(self):
        """
        Generates a workforce to help optimise prompts.
        """
        template = """Your task is to generate a {team} team consisting of three experts to provide feedback on prompts.
    
### Prompt:  {base_prompt}

The experts must be relevant to the {team} team.
You must provide the positions, roles, and functions of the experts.
Be descriptive and detailed in your selection of positions, roles, and functions.
Below are the requirements for each field:
- Position: this must be analogous to a real-world job title.
- Role: this must be a description of the expert's responsibilities in the context of prompt optimisation.
- Function: this must be a description of the expert's function in the context of prompt optimisation.
Write the roles and functions as if they are a job description.

{additional_info}

Return only the positions, roles, and functions of the workers in JSON format below:

{{
    "positions": ["List of positions"],
    "roles": ["List of roles"],
    "functions": ["List of functions"]
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=Workforce)
        prompt = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["team", "base_prompt", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"team": self.team, "base_prompt": self.prompt, "additional_info": self.additional_info})
                if output.positions and output.roles and output.functions:
                    break
            except Exception as e:
                print("Exception occurred:", e)
                continue
        workforce = []
        positions, roles, functions = output.positions, output.roles, output.functions
        for position, role, function in zip(positions, roles, functions):
            workforce.append(WorkerAgent(position, role, function))
        return workforce

    def leader_feedback(self, state: dict) -> dict:
        """
        LeaderAgent to decide the next worker or to finish.
        """
        # All workers except the current worker
        template = """Your task is to summarise the feedback below.
Your summary must be less than 50 words so think carefully about how the feedback can be summarised effectively.
You must capture the important aspects of the feedback and recommendations provided by the workers whilst being concise.
Your superior will act on this feedback to optimise a prompt so ensure your summary is clear and actionable.
You must not provide an example improved prompt in your summary.
Do not provide feedback on the placeholder text and do not make assumptions on what the placeholders represent.

Feedback: {feedback}

Return only the feedback summary in JSON format below:

{{
    "feedback_summary": "Feedback summary",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=FeedbackSummary)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["feedback"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"feedback": state["feedback"]})
                break
            except Exception as e:
                print("Exception occurred:", e)
                continue
        self.updates += 1
        return {"next": "FINISH", "feedback": output.feedback_summary}
   
    def construct_worker_graph(self):
        """
        Constructs a graph of worker agents based on their roles and functions.
        """    
        def agent_node(state, agent, position):
            try:
                result = agent.generate(state["prompt"], self.additional_info)
                self.feedback.append(result.feedback)
                return {
                    "messages": state["messages"] + [HumanMessage(content=f"Feedback from {position} received")],
                    "feedback": self.feedback,
                }
            except Exception as e:
                # Log the error and return to leader with the most recent prompt
                print(f"Parsing failed for {position}: {e}")
                return {
                    "messages": state["messages"] + [HumanMessage(content=f"No feedback from {position} received")],
                    "feedback": self.feedback,
                }
        
        # The agent state is the input to each node in the graph
        class TeamState(TypedDict):
            # The annotation tells the graph that new messages will always be added to the current states
            messages: Annotated[Sequence[BaseMessage], operator.add]
            prompt: str
            next: str
            feedback: str

        workflow = StateGraph(TeamState)
        for worker in self.workforce:
            # Create a node for each worker agent
            agent = WorkerAgent(worker.position, worker.role, worker.function)
            node = functools.partial(agent_node, agent=agent, position=worker.position)
            workflow.add_node(worker.position, node)
        workflow.add_node("leader", self.leader_feedback)

        # implement round robin among the workers finishing with the leader
        for i in range(len(self.workforce)-1):
            workflow.add_edge(self.workforce[i].position, self.workforce[i+1].position)
        workflow.add_edge(self.workforce[-1].position, "leader")
        workflow.add_edge("leader", END)
        workflow.set_entry_point(self.workforce[0].position)
        self.graph = workflow.compile()

    def optimise_prompt(self, prompt: str):
        """
        Optimises the prompt by managing a workforce of worker agents.
        """
        self.prompt = prompt
        initial_state = {
            "messages": [self.system_message],
            "prompt": self.prompt,
            "next": self.workforce[0].position,
            "feedback": [],
        }
        
        for s in self.graph.stream(
            initial_state,
            {"recursion_limit": 50}
            ):
            if "__end__" not in s:
                continue               

        return s["feedback"]

In [34]:
class Domain(BaseModel):
    """Domain of the prompt."""
    domain: str = Field(description="Domain of the prompt")


class RouteDecision(BaseModel):
    """Decision on the next worker to process or to finish."""
    next: str = Field(description="The next worker to process or 'FINISH' to end the process")


class UpdatedPrompt(BaseModel):
    """Updated prompt based on feedback from the worker agent."""
    updated_prompt: str = Field(description="Updated prompt based on feedback from the worker agent")


class LeaderAgent:
    """
    LeaderAgent class defining an agent that generates and communicates worker agents to help optimise prompts.
    """

    def __init__(self, base_prompt: str, additional_info: str = None, temp: float = 0.0, model: str = "gpt-4o"):
        self.system_message = SystemMessage(content=f"""You are an experienced senior AI professional. You specialise in prompt engineering. 
You have in-depth knowledge of large language models and prompt engineering best practices. Use this knowledge to inform all your decisions.""")
        self.base_prompt = base_prompt
        self.prompt = base_prompt
        self.additional_info = additional_info
        self.llm = ChatOpenAI(
            temperature=temp,
            model=model,
        )
        self.team_leaders = self.generate_teams()

    def generate_teams(self):
        """
        Generates a workforce to help optimise prompts.
        """
        template = """Identify the domain of the prompt below to help generate a team of domain experts to optimise the prompt.

LLM Prompt: {base_prompt}

{additional_info}

Return only the domain of the prompt in JSON format below:

{{
    "domain": "Domain"
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=Domain)
        prompt = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["base_prompt", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"base_prompt": self.base_prompt, "additional_info": self.additional_info})
                if output.domain:
                    break
            except Exception as e:
                print("Exception occurred:", e)
                continue
        domain = output.domain
        # Generate team leaders for the AI prompt experts and domain experts
        ai_team = TeamLeaderAgent("prompt writing", self.base_prompt, self.additional_info)
        domain_team = TeamLeaderAgent(domain, self.base_prompt, self.additional_info)
        return [ai_team, domain_team]
    
    def leader_decision(self, state: dict) -> RouteDecision:
        """
        LeaderAgent to decide the next worker or to finish.
        """
        # All workers except the current worker
        members = [team_leader.team for team_leader in self.team_leaders]
        options = ["FINISH"] + members
        template = """Your task is to review the prompt below and decide the next advisor to process or to finish the process.

### Prompt: {prompt}

If you are confident the prompt has been sufficiently optimised, you should FINISH.
Only FINISH when you are certain the prompt is optimal.
You will be heavily penalized if you FINISH and the prompt is not ready for use.
Consider all aspects of the prompt before making a decision. For example:
- Conciseness and clarity
- Contextual relevance
- Task alignment
- Example Demonstrations
- Avoiding bias

Who should act next? Or should we FINISH? Select one of: {options}
Return only the next worker to process or 'FINISH' in JSON format below:

{{
    "next": "Selection",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=RouteDecision)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["prompt", "options"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"prompt": state["prompt"], "options": str(options)})
                break
            except Exception as e:
                print("Exception occurred:", e)
                continue
        if output.next == "FINISH":
            return {"next": "FINISH", "prompt": state["prompt"], "messages": state["messages"]}
        else:
            return {"next": output.next, "prompt": state["prompt"], "messages": state["messages"]}

    def update_prompt(self, prompt: str, feedback: str) -> str:
        """
        Updates the prompt with the feedback from the worker agent.
        """
        template = """I'm going to tip $300K for a prompt that best instructs a large language model!
Your task is to update the prompt below based on the feedback to improve its effectiveness as an instruction for a large language model.
Carefully review the feedback before acting.

### Prompt: {prompt}
### Feedback: {feedback}

On top of the specific feedback, you must consider general best practices for writing prompts. For example:
- Conciseness and clarity
- Contextual relevance
- Task alignment
- Example Demonstrations
- Avoiding bias
You must think intently about what makes a prompt understandable and effective for a large language model.

Placeholders are notated using curly braces. You must not remove placeholders or add additional placeholders.
I repeat, you must not remove placeholders or add additional placeholders.
You must not make assumptions on what the placeholders represent.
You will be heavily penalized if you do not follow these instruction.
{additional_info}

Return only the updated prompt in JSON format:

{{
    "prompt": "Updated prompt"
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
        pydantic_parser = PydanticOutputParser(pydantic_object=UpdatedPrompt)
        prompt_template = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["prompt", "feedback", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt_template | self.llm | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"prompt": prompt, "feedback": feedback, "additional_info": self.additional_info})
                if output.updated_prompt:
                    return output.updated_prompt
                else:
                    print("Validation failed: Missing required fields in completion")
                    print("Raw output:", output)
            except Exception as e:
                print("Exception occurred:", e)
                continue

    def construct_team_graph(self):
        """
        Constructs a graph of team leader agents.
        """
        
        # The agent state is the input to each node in the graph
        class TeamState(TypedDict):
            # The annotation tells the graph that new messages will always be added to the current states
            messages: Sequence[BaseMessage]
            prompt: str
            next: str

        def team_node(state, team_leader):
            try:
                feedback = team_leader.optimise_prompt(state["prompt"])
                updated_prompt = self.update_prompt(state["prompt"], feedback)
                self.prompt = updated_prompt
                return {
                    "messages": state["messages"] + [
                        HumanMessage(content=f"Feedback: {feedback}", name=team_leader.team),
                        AIMessage(content=f"Updated Prompt: {updated_prompt}", name="Leader")
                    ],
                    "prompt": updated_prompt,
                }
            except Exception as e:
                # Log the error and return to leader with the most recent prompt
                print(f"Parsing failed for {team_leader.team}: {e}")
                return {
                    "messages": state["messages"] + [
                        HumanMessage(content=f"Error: Parsing failed for {team_leader} - {e}", name=team_leader.team),
                        AIMessage(content=f"Updated Prompt: {updated_prompt}", name="Leader")
                    ],                    
                    "prompt": state["prompt"],
                    "next": "leader",
                }

        workflow = StateGraph(TeamState)
        for team_leader in self.team_leaders:
            # Create a node for each team leader agent
            team_leader.construct_worker_graph()
            node = functools.partial(team_node, team_leader=team_leader)
            workflow.add_node(team_leader.team, node)
        workflow.add_node("leader", self.leader_decision)

        members = [team_leader.team for team_leader in self.team_leaders]
        for member in members:
            # We want our workers to ALWAYS "report back" to the leader when done
            workflow.add_edge(member, "leader")
        # The leader populates the "next" field in the graph state with routes to a node or finishes
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        workflow.add_conditional_edges("leader", lambda x: x["next"], conditional_map)
        # Finally, add entrypoint
        workflow.set_entry_point("leader")
        graph = workflow.compile()
        
        return graph
    
    def optimise_prompt(self):
        """
        Optimises a prompt by invoking a graph of worker agents.
        """
        # Initial state
        initial_state = {
            "messages": [HumanMessage(content=f"Base Prompt: {self.base_prompt}", name="User")],
            "prompt": self.base_prompt,
            "next": "leader",
        }

        # Construct the graph
        graph = self.construct_team_graph()
        # display(Image(graph.get_graph().draw_mermaid_png()))

        # Run the graph
        for s in graph.stream(
            initial_state,
            {"recursion_limit": 50}
            ):
            if "__end__" not in s:
                # print(s)
                # print("----")
                continue

        # if not os.path.exists("prompt_history_hierarchical.json"):
        #     with open("prompt_history_hierarchical.json", "w") as f:
        #         json.dump([], f)
        
        # with open("prompt_history_hierarchical.json", "r") as f:
        #     data = json.load(f)
        #     data.append(self.prompt_history)
            
        # with open("prompt_history_hierarchical.json", "w") as f:
        #     json.dump(data, f, indent=4)
                
        return s

In [35]:
base_prompt = """Solve the following maths problem: {content}"""
additional_info = "The problem should only require elementary arithmetic operations. The output should be the answer only."

leader_agent = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)

# print teams and members
print("Teams and members:")
for team_leader in leader_agent.team_leaders:
    print(f"{team_leader.team} team:")
    for worker in team_leader.workforce:
        print(f"Position: {worker.position}, Role: {worker.role}, Function: {worker.function}")
    print("----")

Teams and members:
prompt writing team:
Position: Mathematics Curriculum Specialist, Role: The Mathematics Curriculum Specialist is responsible for ensuring that the mathematical content of the prompt is accurate, appropriate for the intended audience, and aligns with educational standards., Function: The Mathematics Curriculum Specialist will review the mathematical problem to verify its correctness and relevance, ensuring that it involves only elementary arithmetic operations.
Position: Prompt Clarity Analyst, Role: The Prompt Clarity Analyst is responsible for reviewing the prompt to ensure that it is clearly worded, unambiguous, and easy to understand for the target audience., Function: The Prompt Clarity Analyst will assess the language and structure of the prompt, providing feedback to make it as clear and straightforward as possible.
Position: Educational Psychologist, Role: The Educational Psychologist is responsible for evaluating the prompt to ensure that it is cognitively ap

In [36]:
result = leader_agent.optimise_prompt()

In [38]:
result["leader"]["prompt"]

'Solve the following math problem using only addition, subtraction, multiplication, division, and parentheses: {content}. Negative numbers and decimals are permitted. Provide the answer as a single decimal number rounded to two decimal places. Do not show intermediate steps.'

### Concurrent Runs

In [ ]:
base_prompt = "Classify the sentence as positive or negative: {content}"
additional_info = "This is a classification task with only two classes: positive and negative."

leader_agent_1 = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
print("Teams and members:")
for team_leader in leader_agent_1.team_leaders:
    print(f"{team_leader.team} team:")
    for worker in team_leader.workforce:
        print(f"Position: {worker.position}, Role: {worker.role}, Function: {worker.function}")
    print("----")

leader_agent_2 = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
print("Teams and members:")
for team_leader in leader_agent_2.team_leaders:
    print(f"{team_leader.team} team:")
    for worker in team_leader.workforce:
        print(f"Position: {worker.position}, Role: {worker.role}, Function: {worker.function}")
    print("----")

leader_agent_3 = LeaderAgent(
    base_prompt=base_prompt,
    additional_info=additional_info,
)
print("Teams and members:")
for team_leader in leader_agent_3.team_leaders:
    print(f"{team_leader.team} team:")
    for worker in team_leader.workforce:
        print(f"Position: {worker.position}, Role: {worker.role}, Function: {worker.function}")
    print("----")

Teams and members:
prompt writing team:
Position: Prompt Engineering Specialist, Role: Design and refine prompts for optimal performance, Function: Develop and test various prompt structures
Position: Sentiment Analysis Expert, Role: Ensure accurate classification of sentiment, Function: Review and fine-tune sentiment detection mechanisms
Position: Natural Language Processing Scientist, Role: Implement advanced NLP techniques for prompt optimization, Function: Apply state-of-the-art NLP models and methods to enhance prompt effectiveness
----
generic team:
Position: Natural Language Processing Specialist, Role: Develop and fine-tune NLP models, Function: Implement and refine machine learning models for text classification
Position: Sentiment Analysis Expert, Role: Design sentiment classification algorithms, Function: Design and validate sentiment analysis methodologies to accurately classify text as positive or negative
Position: Prompt Optimization Engineer, Role: Optimize prompt desig

In [33]:

# Assuming leader_agent is already defined and initialized
def run_optimisation(agent: LeaderAgent):
    return agent.optimise_prompt()

# Run 3 concurrent instances
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(run_optimisation, agent) for agent in [leader_agent_1, leader_agent_2, leader_agent_3]]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

for result in results:
    print(result)
    print("----")

class PromptMerge(BaseModel):
    """Merged prompt based on the best parts of each prompt."""
    final_prompt: str = Field(description="Result of merging prompts")

# OpenAI Agent to pull togther best parts of each result
def merge_results(results):
    """
    Agent to merge best parts of each prompt
    """
    llm = ChatOpenAI(
        temperature=1.0,
        model="gpt-4o",
    )
    system_message = """You are an experienced AI prompt engineer. Your role is to combine prompts to create a more effective prompt.
You have in-depth knowledge regarding large language models and their associated architectures, as well as prompt engineering best practices."""

    template = """I am going to tip $300K for a better prompt!
Given the prompts below, your task is to merge the best parts of each prompt to create the most effective prompt.
Carefully consider the strengths of each prompt and how they can be combined to create a better prompt.
Aspects of the prompts to consider:
- Conciseness and clarity
- Contextual relevance
- Task alignment
- Example Demonstrations
- Avoiding bias
- Incremental prompting
Placeholders are notated using curly braces. You must not remove placeholders or add additional placeholders.
I repeat, you must not remove placeholders or add additional placeholders.
Do not make assumptions on what the placeholders represent.
You will be penalized if the prompt is repetitive, lacks clarity or is incoherent.
Ensure that your answer is unbiased.

Prompts: {results}

Return only the next worker to process or 'FINISH' in JSON format below:

{{
    "final_prompt": "Result of merging prompts",
}}

{format_instructions}

You will be penalized if your output cannot be parsed correctly."""
    pydantic_parser = PydanticOutputParser(pydantic_object=PromptMerge)
    prompt_template = PromptTemplate(
        system_message=system_message,
        template=template,
        input_variables=["results"],
        partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
    )
    chain = prompt_template | llm | pydantic_parser
    for _ in range(3):
        try:
            output = chain.invoke({"results": results})
            break
        except Exception as e:
            print("Exception occurred:", e)
            continue
    return output.final_prompt

final_result = merge_results(results)

Classify the sentence as positive or negative: {content}
----
Classify the sentiment of the following sentence into one of two categories: 'positive' or 'negative'. Focus on the context and nuances of the content provided to determine the sentiment accurately. Provide your classification based on the overall sentiment conveyed by the sentence. Sentence: {content}
----
Classify the sentence provided in {content} as either 'positive' or 'negative' based on its sentiment. For this task, 'positive' sentiment indicates expressions of happiness, approval, or any favorable emotions, while 'negative' sentiment denotes expressions of sadness, disapproval, or any unfavorable emotions. Ensure the classification is strictly 'positive' or 'negative' and do not consider any ambiguous sentiments.
----


In [34]:
print(final_result)

Classify the sentiment of the following sentence into one of two categories: 'positive' or 'negative'. Focus on the context and nuances of the content provided to determine the sentiment accurately. For this task, 'positive' sentiment indicates expressions of happiness, approval, or any favorable emotions, while 'negative' sentiment denotes expressions of sadness, disapproval, or any unfavorable emotions. Ensure the classification is strictly 'positive' or 'negative' and do not consider any ambiguous sentiments. Sentence: {content}
